In [4]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import copy

class GameState:
    def __init__(self, board=None, player_pieces=None, ai_pieces=None):
        if board is not None:
            self.board = np.copy(board)
        else:
            self.board = np.array([
                ['A', 'X', '0', 'B'],
                ['A', '0', '0', 'B'],
                ['A', '0', 'X', 'B']
            ])
        if player_pieces is not None:
            self.player_pieces = player_pieces.copy()
        else:
            self.player_pieces = [(0, 0), (1, 0), (2, 0)]
        if ai_pieces is not None:
            self.ai_pieces = ai_pieces.copy()
        else:
            self.ai_pieces = [(0, 3), (1, 3), (2, 3)]

    def get_valid_moves(self, piece_positions, player_type):
        moves = {}
        for piece in piece_positions:
            x, y = piece
            possible_moves = []
            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < 3 and 0 <= ny < 4:
                    if self.board[nx][ny] == '0':
                        # Evitar mover a las posiciones objetivo del otro jugador
                        if player_type == 'AI' and (nx, ny) in [(0, 0), (1, 0), (2, 0)] and self.board[x][y] != 'B':
                            continue
                        possible_moves.append((nx, ny))
            if possible_moves:
                moves[piece] = possible_moves
        return moves

    def move_piece(self, from_pos, to_pos, player_type):
        x1, y1 = from_pos
        x2, y2 = to_pos
        symbol = 'A' if player_type == 'Player' else 'B'
        self.board[x2][y2] = symbol
        self.board[x1][y1] = '0'
        if player_type == 'Player':
            self.player_pieces.remove(from_pos)
            self.player_pieces.append((x2, y2))
        else:
            self.ai_pieces.remove(from_pos)
            self.ai_pieces.append((x2, y2))

    def check_winner(self):
        player_goal_positions = [(0, 3), (1, 3), (2, 3)]
        ai_goal_positions = [(0, 0), (1, 0), (2, 0)]
        # Verificar si todas las posiciones objetivo están ocupadas por las fichas del jugador correspondiente
        if all(self.board[pos] == 'A' for pos in player_goal_positions):
            return 'Player'
        if all(self.board[pos] == 'B' for pos in ai_goal_positions):
            return 'AI'
        return None

    def heuristic(self):
        ai_goal_positions = [(0, 0), (1, 0), (2, 0)]
        player_goal_positions = [(0, 3), (1, 3), (2, 3)]

        # Penalizar si las posiciones objetivo de la IA están ocupadas por el jugador
        occupied_ai_goals_by_player = sum(1 for pos in ai_goal_positions if self.board[pos] == 'A')
        score = -occupied_ai_goals_by_player * 10

        # Bonificar por cada posición objetivo de la IA que esté libre
        free_ai_goal_positions = sum(1 for pos in ai_goal_positions if self.board[pos] == '0')
        score += free_ai_goal_positions * 50

        # Penalizar si las fichas del jugador están bloqueadas en las posiciones objetivo de la IA
        for pos in ai_goal_positions:
            if self.board[pos] == 'A':
                player_moves = self.get_valid_moves([pos], 'Player')
                if not player_moves:
                    score -= 5000

        # Priorizar el avance de la IA
        ai_distance = sum([abs(pos[1] - 0) for pos in self.ai_pieces])
        score -= ai_distance * 100

        # Añadir una pequeña penalización por la distancia del jugador al objetivo
        player_distance = sum([abs(pos[1] - 3) for pos in self.player_pieces])
        score += player_distance * 10

        return score

    def copy(self):
        return GameState(self.board, self.player_pieces, self.ai_pieces)

class GomokuGameGUI:
    def __init__(self):
        self.game_state = GameState()
        self.turn = 'Player'  # 'Player' or 'AI'
        self.game_over = False
        self.selected_piece = None
        self.info_label = widgets.Label(value="Tu turno (A)")
        display(self.info_label)
        self.create_board()

    def create_board(self):
        self.buttons = []
        for i in range(3):
            row = []
            for j in range(4):
                value = self.game_state.board[i][j]
                button = widgets.Button(description=value, layout=widgets.Layout(width='60px', height='60px'))
                button.style.button_color = self.get_button_color(value)
                button.on_click(self.on_button_click(i, j))
                row.append(button)
            self.buttons.append(row)
        self.board_ui = widgets.GridBox(
            children=[button for row in self.buttons for button in row],
            layout=widgets.Layout(
                width='260px',
                grid_template_columns='repeat(4, 60px)',
                grid_template_rows='repeat(3, 60px)'
            )
        )
        display(self.board_ui)

    def get_button_color(self, value):
        if value == 'A':
            return 'lightblue'
        elif value == 'B':
            return 'salmon'
        elif value == 'X':
            return 'black'
        else:
            return 'lightgray'

    def on_button_click(self, row, col):
        def handle_click(button):
            if self.game_over or self.turn != 'Player':
                return
            if self.selected_piece is None:
                # Seleccionar una ficha del jugador
                if self.game_state.board[row][col] == 'A':
                    self.selected_piece = (row, col)
                    button.style.button_color = 'yellow'
                    self.info_label.value = f"Has seleccionado la ficha en {self.selected_piece}. Elige dónde moverla."
            else:
                # Intentar mover la ficha seleccionada a la posición clicada
                from_pos = self.selected_piece
                to_pos = (row, col)
                if self.is_valid_move(from_pos, to_pos):
                    self.game_state.move_piece(from_pos, to_pos, 'Player')
                    self.update_board_ui()
                    self.selected_piece = None
                    winner = self.game_state.check_winner()
                    if winner:
                        self.game_over = True
                        self.info_label.value = "¡Has ganado!"
                        self.end_game()
                    else:
                        # Verificar si la IA está bloqueada
                        if not self.game_state.get_valid_moves(self.game_state.ai_pieces, 'AI'):
                            self.info_label.value = "La IA no tiene movimientos. Tienes un movimiento adicional."
                            if not self.game_state.get_valid_moves(self.game_state.player_pieces, 'Player'):
                                self.info_label.value = "Ambos jugadores están bloqueados. Es un empate."
                                self.game_over = True
                                self.end_game()
                            else:
                                self.turn = 'Player'
                        else:
                            self.turn = 'AI'
                            self.info_label.value = "Turno de la IA"
                            self.ai_turn()
                else:
                    self.info_label.value = "Movimiento inválido. Intenta de nuevo."
                    self.reset_selection()
        return handle_click

    def reset_selection(self):
        self.selected_piece = None
        self.update_board_ui()
        self.info_label.value = "Tu turno (A). Selecciona una ficha para mover."

    def is_valid_move(self, from_pos, to_pos):
        if from_pos not in self.game_state.player_pieces:
            return False
        x1, y1 = from_pos
        x2, y2 = to_pos
        if abs(x1 - x2) + abs(y1 - y2) != 1:
            return False
        if self.game_state.board[x2][y2] != '0':
            return False
        return True

    def update_board_ui(self):
        for i in range(3):
            for j in range(4):
                value = self.game_state.board[i][j]
                button = self.buttons[i][j]
                button.description = value
                button.style.button_color = self.get_button_color(value)
                button.disabled = self.game_over
        if self.selected_piece:
            x, y = self.selected_piece
            self.buttons[x][y].style.button_color = 'yellow'

    def ai_turn(self):
        while True:
            valid_moves = self.game_state.get_valid_moves(self.game_state.ai_pieces, 'AI')
            if not valid_moves:
                self.info_label.value = "La IA no tiene movimientos. Tu turno nuevamente."
                if not self.game_state.get_valid_moves(self.game_state.player_pieces, 'Player'):
                    self.info_label.value = "Ambos jugadores están bloqueados. Es un empate."
                    self.game_over = True
                    self.end_game()
                else:
                    self.turn = 'Player'
                return
            best_score = -float('inf')
            best_moves = []
            alpha = -float('inf')
            beta = float('inf')
            for from_pos, moves in valid_moves.items():
                for to_pos in moves:
                    temp_state = self.game_state.copy()
                    temp_state.move_piece(from_pos, to_pos, 'AI')
                    score = self.minimax(temp_state, 10, False, alpha, beta)
                    if score > best_score:
                        best_score = score
                        best_moves = [(from_pos, to_pos)]
                        alpha = max(alpha, best_score)
                    elif score == best_score:
                        best_moves.append((from_pos, to_pos))
            if best_moves:
                # Elegir un movimiento al azar entre los mejores
                best_move = best_moves[np.random.randint(len(best_moves))]
                self.game_state.move_piece(best_move[0], best_move[1], 'AI')
                self.update_board_ui()
                winner = self.game_state.check_winner()
                if winner:
                    self.game_over = True
                    self.info_label.value = "La IA ha ganado."
                    self.end_game()
                    return
                else:
                    if not self.game_state.get_valid_moves(self.game_state.player_pieces, 'Player'):
                        self.info_label.value = "No tienes movimientos. La IA tiene un movimiento adicional."
                        if not self.game_state.get_valid_moves(self.game_state.ai_pieces, 'AI'):
                            self.info_label.value = "Ambos jugadores están bloqueados. Es un empate."
                            self.game_over = True
                            self.end_game()
                            return
                        else:
                            continue  # La IA obtiene un movimiento adicional
                    else:
                        self.turn = 'Player'
                        self.info_label.value = "Tu turno (A)"
                        return
            else:
                self.info_label.value = "La IA no pudo realizar un movimiento."
                self.turn = 'Player'
                self.info_label.value = "Tu turno (A)"
                return

    def minimax(self, game_state, depth, is_maximizing, alpha, beta):
        winner = game_state.check_winner()
        if winner == 'Player':
            return -10000 - depth
        elif winner == 'AI':
            return 10000 + depth
        if depth == 0:
            return game_state.heuristic()
        if is_maximizing:
            max_eval = -float('inf')
            valid_moves = game_state.get_valid_moves(game_state.ai_pieces, 'AI')
            if not valid_moves:
                return game_state.heuristic()
            for from_pos, moves in valid_moves.items():
                for to_pos in moves:
                    temp_state = game_state.copy()
                    temp_state.move_piece(from_pos, to_pos, 'AI')
                    eval = self.minimax(temp_state, depth - 1, False, alpha, beta)
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
            return max_eval
        else:
            min_eval = float('inf')
            valid_moves = game_state.get_valid_moves(game_state.player_pieces, 'Player')
            if not valid_moves:
                return game_state.heuristic()
            for from_pos, moves in valid_moves.items():
                for to_pos in moves:
                    temp_state = game_state.copy()
                    temp_state.move_piece(from_pos, to_pos, 'Player')
                    eval = self.minimax(temp_state, depth - 1, True, alpha, beta)
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
            return min_eval

    def end_game(self):
        for i in range(3):
            for j in range(4):
                self.buttons[i][j].disabled = True
        reset_button = widgets.Button(description='Reiniciar Juego', layout=widgets.Layout(width='260px'))
        reset_button.on_click(self.reset_game)
        display(reset_button)

    def reset_game(self, button):
        clear_output()
        self.__init__()

# Ejecutamos el juego
game = GomokuGameGUI()


Label(value='Tu turno (A)')

GridBox(children=(Button(description='A', layout=Layout(height='60px', width='60px'), style=ButtonStyle(button…